In [295]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

## Run Vivado

In [286]:
modules = os.listdir("Modules")

In [287]:
modules

['.ipynb_checkpoints',
 'top_CLA8.v',
 'top_CLA16.v',
 'top_CLA64.v',
 'top_CLA32.v',
 'PGGen.v']

In [3]:
top_modules = [mod for mod in modules if mod[:3]=='top']

In [4]:
top_modules

['top_CLA8.v', 'top_CLA16.v', 'top_CLA64.v', 'top_CLA32.v']

In [5]:
os.system("vivado -mode batch -source Script/tcl_add.tcl")

0

In [6]:
for filename in top_modules:
    os.mkdir("Impl/"+filename[:-2])
    #print(filename[:-2])
    os.system("vivado -mode batch -source Script/tcl_run.tcl -tclargs {}".format(filename))

top_CLA8
top_CLA16
top_CLA64
top_CLA32


In [7]:
top_modules[0][:-2]

'top_CLA8'

## Analyse Metric

In [271]:
designs = os.listdir("Impl")
file = ["power.txt", "timing.txt","utilization.txt"]
col = ["Signal Power(w)","Logic Power(w)" ,"Static Power(w)", "DC Power(w)", "Total Power(w)","DataPath Delay", "F7_Muxes","LUT_as_memory", "LUT_as_logic","Slice_LUTs","LUT1","LUT2","LUT3","LUT4","LUT5","LUT6","LUT7","LUT_as_memory","LUT_as_logic2","Slice_LUTs2","LUTs_difference"]

In [272]:
#col

In [297]:
# tonpdes = []
Sig_Pow_des = []
Logic_Power_des=[]
Static_Power_des=[]
DC_Power_des=[]
Total_Power_des=[]
dpddes = []
F7_Muxes_des=[]
LUT_as_memory_des=[]
LUT_as_memory_des2=LUT_as_memory_des
LUT_as_logic_des=[]
sldes = []
LUT1_des=[]
LUT2_des=[]
LUT3_des=[]
LUT4_des=[]
LUT5_des=[]
LUT6_des=[]
LUT7_des=[]
LS_des=[]
Slice_LUTs2_des=LS_des
LUTs_difference_des=[]
for des in designs:
    LS=0
    for fl in file:
        with open("Impl/{}/{}".format(des,fl)) as f:
            lines = f.read()
            
        sp = lines.split("\n")
        
        for l in sp:
            if fl == "power.txt":
                # if "Total On-Chip Power" in l:
                #     tonp = float(l.split()[6])
                #     tonpdes.append(tonp)
                if "Signals" in l:
                    Sig_Pow = float(l.split()[3])
                    Sig_Pow_des.append(Sig_Pow)
                if "Slice Logic" in l:
                    Logic_Power = float(l.split()[4])
                    Logic_Power_des.append(Logic_Power)
                if "Static Power" in l:
                    Static_Power = float(l.split()[4])
                    Static_Power_des.append(Static_Power)
                if "Total On-Chip Power" in l:
                    Total_Power = float(l.split()[6])
                    Total_Power_des.append(Total_Power)
                    DC_Power = Total_Power-Static_Power
                    DC_Power_des.append(DC_Power)
                
                    
            if fl == "timing.txt":
                if "Data Path Delay" in l:
                    dpd = l.split()[3]
                    dpddes.append(dpd)
            
            if fl == "utilization.txt":  
                if "F7 Muxes" in l:
                    FM = int(l.split()[4])
                    F7_Muxes_des.append(FM)
                    
                
                if "| LUT as Memory" in l:
                    LUT_Mem = int(l.split()[5])
                    LUT_as_memory_des.append(LUT_Mem)
               
                
                if "| LUT as Logic " in l:
                    LUT_Logic = int(l.split()[5])
                    LUT_as_logic_des.append(LUT_Logic) 
                
                if "Slice LUTs" in l:
                    sl = int(l.split()[4])
                    sldes.append(sl)   
                
                if "LUT1" in l:
                    lut1 = int(l.split()[5])
                    LUT1_des.append(lut1)
                    LS=LS+lut1
                    
                if "| LUT2" in l:
                    lut2 = int(l.split()[3])
                    LUT2_des.append(lut2)
                    LS=LS+lut2
                
                if "| LUT3" in l:
                    lut3 = int(l.split()[3])
                    LUT3_des.append(lut3)
                    LS=LS+lut3
                
                if "| LUT4" in l:
                    lut4 = int(l.split()[3])
                    LUT4_des.append(lut4)
                    LS=LS+lut4
                    
                if "| LUT5" in l:
                    lut5 = int(l.split()[3])
                    LUT5_des.append(lut5)
                    LS=LS+lut5
                
                if "| LUT6" in l:
                    lut6 = int(l.split()[3])
                    LUT6_des.append(lut6)
                    LS=LS+lut6
                
                if "| LUT7" in l:
                    lut2 = int(l.split()[3])
                    LUT7_des.append(lut7)
                    LS=LS+lut7
                    
    diff=LS-sl
    LS_des.append(LS)
    LUTs_difference_des.append(diff)
    
               
            

In [298]:
print(LS_des)

[15, 45, 286, 123]


In [299]:
result = [Sig_Pow_des, Logic_Power_des, Static_Power_des, DC_Power_des, Total_Power_des,dpddes, F7_Muxes_des, LUT_as_memory_des, LUT_as_logic_des,sldes,LUT1_des,LUT2_des,LUT3_des,LUT4_des,LUT5_des,LUT6_des,LUT7_des,LUT_as_memory_des2,LS_des,Slice_LUTs2_des,LUTs_difference_des]
df = pd.DataFrame(result)
df=df.fillna(0)
df_t = df.T
df_t.columns = col 
df_t.insert(loc=0, column='File Name', value=designs)
df.columns = designs
df.insert(loc=0, column='Metric', value=col)
df.to_csv("Result/Result1.csv",index=None)
df_t.to_csv("Result/Result2.csv",index=None)
df_t

,File Name,Signal Power(w),Logic Power(w),Static Power(w),DC Power(w),Total Power(w),DataPath Delay,F7_Muxes,LUT_as_memory,LUT_as_logic,...,LUT2,LUT3,LUT4,LUT5,LUT6,LUT7,LUT_as_memory,LUT_as_logic2,Slice_LUTs2,LUTs_difference
0,top_CLA8,0.17,0.066,0.191,4.488,5.002,6.927ns,0,0,13,...,1,2,3,3,6,0,0,15,15,2
1,top_CLA16,0.496,0.192,0.256,10.95,11.141,8.335ns,0,0,35,...,6,4,8,5,22,0,0,45,45,10
2,top_CLA64,4.872,1.148,2.405,48.996,49.252,16.020ns,0,0,233,...,48,14,25,44,155,0,0,286,286,53
3,top_CLA32,1.724,0.507,0.514,20.814,23.219,11.285ns,0,0,97,...,26,9,14,15,59,0,0,123,123,26


In [276]:
 for des in designs:
    for fl in file:
        with open("Impl/{}/{}".format(des,fl)) as f:
            lines = f.read()
            
        sp = lines.split("\n")
        
        for l in sp: 
            if "Data Path Delay" in l:
                    dpd = l.split()[3]
                    dpddes.append(dpd)
                   # print(l)
                    print(dpd)
#             if "Slice LUTs" in l:
#                    # LUT_Mem = int(l.split()[4])
#                    # LUT_as_memory_des.append(LUT_MEM)
#                     print(l)
#                     print(l.split()[4])
#                    # sl = int(l.split()[-4])
#                    # sldes.append(sl) 
#                    # Sig_Pow = float(l.split()[3])
#                    # Sig_Pow_des.append(Sig_Pow)

6.927ns
8.335ns
16.020ns
11.285ns
